# ETL for the Polk 2016 Snow Emergency Tows Data Set
The .CSV file for the Polk 2016 Snow Emergency does not contain the latitudes and longitudes of the towing incidents.  The GeoJSON file does include this information.  We can use a GeoPandas GeoDataFrame to manipulate this GeoJSON to extract the longitude and latitude information.  We perform the following steps:

1.  Fill missing coordinates by using Google Places to geocode the given address information.
2.  Fill missing Ward, Community, and Neighborhood information in using shapely's `polygon.contains(point)` functionality.  We have GeoJSONs with the boundaries for Minneapolis Wards, Communities, and Neighborhoods.  For each point, we can write a function that returns the corresponding Ward, Community, and Neighborhood.
3.  Extract the GeoDataFrame to an ordinary Pandas DataFrame by dropping the geometry column to a pair of columns: latitude and longitude.
4.  Split the datetime string into a date and time field.
5.  Write the final data frame to a `.csv` file to be combined with the other cleaned files.

In [1]:
# import the dependencies
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import geopy

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from geopy.geocoders import GoogleV3

from api_keys import api_key

In [7]:
# constants related to input and output

DATA_DIR = "data"
POLK_FILE = "Snow_Emergency_Polk_Tows_2016.geojson"
OUTPUT_DIR = "output"
WARDS_FILE = "City_Council_Wards.geojson"
COMMUNITIES_FILE = "Communities.geojson"
NEIGHBORHOODS_FILE = "Minneapolis_Neighborhoods.geojson"
OUTPUT_FILE = "Polk_Tows.csv"

In [8]:
# a helper function to read files in.  GeoPandas read_file can load GeoJSONs

def load_data(filename, data_dir=DATA_DIR):
    filepath = os.path.join(data_dir, filename)
    return gpd.read_file(filepath)

In [9]:
polk_tows = load_data(POLK_FILE)
polk_tows.head()

,FID,X,Y,Call_Taken,Location,Day,geometry
0,1001,0.000000e+00,0.000000e+00,2016-02-04T00:00:00,2213 N 4th st,Day 3,None
1,1002,-1.038496e+07,5.619481e+06,2016-02-04T00:00:00,821 Lyndale PL,Day 3,POINT (-93.28965794887922 44.98703965146753)
2,1003,-1.038620e+07,5.612199e+06,2016-02-04T00:00:00,3431 Humboldt Ave South,Day 3,POINT (-93.30085220376344 44.94075481079494)
3,1004,-1.038058e+07,5.619614e+06,2016-02-04T00:00:00,521 3rd Ave se,Day 3,POINT (-93.25038048234984 44.98788273781837)
4,1005,-1.038060e+07,5.619218e+06,2016-02-04T00:00:00,321 4th ave se,Day 3,POINT (-93.25053762624376 44.98536724219078)


In [10]:
polk_tows.shape

(1480, 7)

# 1.  Fill in Missing Geometries

In [11]:
# how many observations are missing geometry?
polk_tows.geometry.isnull().sum()

100

In [12]:
# prepare to geocode missing geometries:
geolocator = GoogleV3(api_key=api_key)

In [16]:
# Loop over the GeoDataFrame, if the geometry is missing geocode from the Location
import time

for idx in range(polk_tows.shape[0]):
    if not polk_tows.loc[idx, 'geometry']:
        address = polk_tows.loc[idx, 'Location'] + ", Minneapolis, MN"
        print(address)
        result =  geolocator.geocode(address)
        geom = Point(result.longitude, result.latitude)
        time.sleep(2) # avoid the API rate limit
        polk_tows.loc[idx, 'geometry'] = geom

Bryant and lyndale, Minneapolis, MN
917russell ave n, Minneapolis, MN
111 logan Ave n, Minneapolis, MN
2325 plilsbury ave s, Minneapolis, MN
3953 bupout Ave n, Minneapolis, MN
2615 Essex Sr se, Minneapolis, MN
1829 Stevens south, Minneapolis, MN
1731 s 2nd ave, Minneapolis, MN
205 7 TN Ave se, Minneapolis, MN
22nd and bryant, Minneapolis, MN
1009 w 37th st, Minneapolis, MN
330 w 15th st, Minneapolis, MN
113 w 15th st, Minneapolis, MN
13th Y NE 100-123, Minneapolis, MN
Cty rd 5, Minneapolis, MN
1769 13th Ave NE, Minneapolis, MN
4337 blooming ave s, Minneapolis, MN
600 N 2ND ST, Minneapolis, MN
Marshall St & 3rd Ave NE, Minneapolis, MN
1769 13th Ave NE, Minneapolis, MN
15th AVe 8th st SE, Minneapolis, MN
3331 henn ave s, Minneapolis, MN
2815 e lake isles pkwy, Minneapolis, MN
2833 e llake of isles, Minneapolis, MN
410 N 6th ave, Minneapolis, MN
3108 N 4th st, Minneapolis, MN
1050 Van White Blvd, Minneapolis, MN
3115 DuPont ace s, Minneapolis, MN
2416 N 4th st, Minneapolis, MN
8th st se n

In [17]:
polk_tows.geometry.isnull().sum()

0

In [18]:
# Save the work done so far to avoid needing to geocode again:
polk_tows.to_csv(os.path.join(OUTPUT_DIR, "grant_temp.csv"))

# 2.  Determine Wards, Communities, Neighborhoods

In [19]:
wards = load_data(WARDS_FILE)
wards.head()

,FID,BDNUM,Shape_STAr,Shape_STLe,geometry
0,1,1,2.044159e+08,71797.801731,"POLYGON ((-93.2268508021494 45.0132152671148, ..."
1,2,9,7.290147e+07,42091.446370,"POLYGON ((-93.2431848712514 44.955829056264, -..."
2,3,3,1.062057e+08,57104.348729,"POLYGON ((-93.2631316871618 45.0131674841788, ..."
3,4,4,1.471704e+08,57462.048968,"POLYGON ((-93.2991995476941 45.0511367258392, ..."
4,5,8,7.869230e+07,47568.406676,"POLYGON ((-93.2747782291209 44.9483536669077, ..."


In [20]:
# A helper function for determining the ward of a point:

def find_ward(place, wards_df=wards):
    for row in wards_df.itertuples():
        if row.geometry.contains(place):
            return row.BDNUM


In [21]:
polk_tows['Ward'] = polk_tows.geometry.map(find_ward)

In [22]:
polk_tows.head()

,FID,X,Y,Call_Taken,Location,Day,geometry,Ward
0,1001,0.000000e+00,0.000000e+00,2016-02-04T00:00:00,2213 N 4th st,Day 3,POINT (-93.28554899999999 44.999581),5
1,1002,-1.038496e+07,5.619481e+06,2016-02-04T00:00:00,821 Lyndale PL,Day 3,POINT (-93.28965794887922 44.98703965146753),5
2,1003,-1.038620e+07,5.612199e+06,2016-02-04T00:00:00,3431 Humboldt Ave South,Day 3,POINT (-93.30085220376344 44.94075481079494),10
3,1004,-1.038058e+07,5.619614e+06,2016-02-04T00:00:00,521 3rd Ave se,Day 3,POINT (-93.25038048234984 44.98788273781837),3
4,1005,-1.038060e+07,5.619218e+06,2016-02-04T00:00:00,321 4th ave se,Day 3,POINT (-93.25053762624376 44.98536724219078),3


In [23]:
communities = load_data(COMMUNITIES_FILE)
communities.head()

,FID,CommName,geometry
0,1,Camden,"POLYGON ((-93.31949186501051 45.0512462469094,..."
1,2,Northeast,"POLYGON ((-93.22685080614011 45.0132165472863,..."
2,3,Near North,"POLYGON ((-93.3178372365291 45.013260021455, -..."
3,4,Central,"POLYGON ((-93.27211391203549 44.9921313605316,..."
4,5,University,"POLYGON ((-93.2076071706838 45.0015313002637, ..."


In [24]:
# A helper function for determining the community of a point:

def find_community(place, community_df=communities):
    for row in community_df.itertuples():
        if row.geometry.contains(place):
            return row.CommName


In [25]:
polk_tows['Community'] = polk_tows.geometry.map(find_community)

In [26]:
polk_tows.head()

,FID,X,Y,Call_Taken,Location,Day,geometry,Ward,Community
0,1001,0.000000e+00,0.000000e+00,2016-02-04T00:00:00,2213 N 4th st,Day 3,POINT (-93.28554899999999 44.999581),5,Near North
1,1002,-1.038496e+07,5.619481e+06,2016-02-04T00:00:00,821 Lyndale PL,Day 3,POINT (-93.28965794887922 44.98703965146753),5,Near North
2,1003,-1.038620e+07,5.612199e+06,2016-02-04T00:00:00,3431 Humboldt Ave South,Day 3,POINT (-93.30085220376344 44.94075481079494),10,Calhoun Isle
3,1004,-1.038058e+07,5.619614e+06,2016-02-04T00:00:00,521 3rd Ave se,Day 3,POINT (-93.25038048234984 44.98788273781837),3,University
4,1005,-1.038060e+07,5.619218e+06,2016-02-04T00:00:00,321 4th ave se,Day 3,POINT (-93.25053762624376 44.98536724219078),3,University


In [27]:
neighborhoods = load_data(NEIGHBORHOODS_FILE)
neighborhoods.head()

,FID,BDNAME,BDNUM,TEXT_NBR,Shape_STAr,Shape_STLe,NCR_LINK,IMAGE,geometry
0,1,Phillips West,90,90,1.066925e+07,14403.885934,http://www.nrp.org/r2/Neighborhoods/Orgs/PHW.html,PHW,"POLYGON ((-93.2625807586419 44.9609082137146, ..."
1,2,Downtown West,87,87,2.075613e+07,19220.602541,http://www.nrp.org/r2/Neighborhoods/Orgs/DTN.html,DTN,"POLYGON ((-93.2601055025157 44.9829952758614, ..."
2,3,Downtown East,88,88,1.025499e+07,13436.601356,http://www.nrp.org/r2/Neighborhoods/Orgs/DTN.html,DTN,"POLYGON ((-93.2449864570206 44.9789336625517, ..."
3,4,Ventura Village,89,89,1.263526e+07,16988.532717,http://www.nrp.org/r2/Neighborhoods/Orgs/VEN.html,VEN,"POLYGON ((-93.24957700344829 44.9662967560422,..."
4,5,Sumner - Glenwood,29,29,5.741860e+06,11065.343364,http://www.nrp.org/r2/Neighborhoods/Orgs/SGL.html,SGL,"POLYGON ((-93.2882976528817 44.9890356035354, ..."


In [28]:
neighborhoods.shape

(87, 9)

In [29]:
# A helper function for determining the neighborhood of a point:

def find_neighborhood(place, neighborhood_df=neighborhoods):
    for row in neighborhood_df.itertuples():
        if row.geometry.contains(place):
            return row.BDNAME

In [30]:
polk_tows['Neighborhood'] = polk_tows.geometry.map(find_neighborhood)
polk_tows.head()

,FID,X,Y,Call_Taken,Location,Day,geometry,Ward,Community,Neighborhood
0,1001,0.000000e+00,0.000000e+00,2016-02-04T00:00:00,2213 N 4th st,Day 3,POINT (-93.28554899999999 44.999581),5,Near North,Hawthorne
1,1002,-1.038496e+07,5.619481e+06,2016-02-04T00:00:00,821 Lyndale PL,Day 3,POINT (-93.28965794887922 44.98703965146753),5,Near North,Sumner - Glenwood
2,1003,-1.038620e+07,5.612199e+06,2016-02-04T00:00:00,3431 Humboldt Ave South,Day 3,POINT (-93.30085220376344 44.94075481079494),10,Calhoun Isle,ECCO
3,1004,-1.038058e+07,5.619614e+06,2016-02-04T00:00:00,521 3rd Ave se,Day 3,POINT (-93.25038048234984 44.98788273781837),3,University,Marcy Holmes
4,1005,-1.038060e+07,5.619218e+06,2016-02-04T00:00:00,321 4th ave se,Day 3,POINT (-93.25053762624376 44.98536724219078),3,University,Marcy Holmes


In [31]:
# Save the work done so far to avoid needing to geocode again:
polk_tows.to_csv(os.path.join(OUTPUT_DIR, "polk_temp.csv"))

# 3.  Drop the geometry column.  Add two columns for Longitude and Latitude.

In [32]:
def point_longitude (point):
    return point.x

def point_latitude (point):
    return point.y

polk_tows['Longitude'] = polk_tows['geometry'].x
polk_tows['Latitude'] = polk_tows['geometry'].y
polk_tows.head()

,FID,X,Y,Call_Taken,Location,Day,geometry,Ward,Community,Neighborhood,Longitude,Latitude
0,1001,0.000000e+00,0.000000e+00,2016-02-04T00:00:00,2213 N 4th st,Day 3,POINT (-93.28554899999999 44.999581),5,Near North,Hawthorne,-93.285549,44.999581
1,1002,-1.038496e+07,5.619481e+06,2016-02-04T00:00:00,821 Lyndale PL,Day 3,POINT (-93.28965794887922 44.98703965146753),5,Near North,Sumner - Glenwood,-93.289658,44.987040
2,1003,-1.038620e+07,5.612199e+06,2016-02-04T00:00:00,3431 Humboldt Ave South,Day 3,POINT (-93.30085220376344 44.94075481079494),10,Calhoun Isle,ECCO,-93.300852,44.940755
3,1004,-1.038058e+07,5.619614e+06,2016-02-04T00:00:00,521 3rd Ave se,Day 3,POINT (-93.25038048234984 44.98788273781837),3,University,Marcy Holmes,-93.250380,44.987883
4,1005,-1.038060e+07,5.619218e+06,2016-02-04T00:00:00,321 4th ave se,Day 3,POINT (-93.25053762624376 44.98536724219078),3,University,Marcy Holmes,-93.250538,44.985367


# 4.  Convert Call_Taken to a Date String and Time String.  Note that the Times are all 00:00:00

In [33]:
import dateutil.parser as dparser
from datetime import datetime

test = dparser.parse(polk_tows.loc[0, 'Call_Taken'], fuzzy=True)
print(test.strftime("%m/%d/%Y"))
print(test.strftime("%H:%M"))

02/04/2016
00:00


In [34]:
# A helper function to map against the `Call_Taken` column to get the date

def get_date(call_string):
    call_dt = dparser.parse(call_string, fuzzy=True)
    return call_dt.strftime("%m/%d/%Y")

def get_time(call_string):
    call_dt = dparser.parse(call_string, fuzzy=True)
    return call_dt.strftime("%H:%M")

polk_tows['Date'] = polk_tows.Call_Taken.map(get_date)
polk_tows['Time'] = polk_tows.Call_Taken.map(get_time)
polk_tows.head()

,FID,X,Y,Call_Taken,Location,Day,geometry,Ward,Community,Neighborhood,Longitude,Latitude,Date,Time
0,1001,0.000000e+00,0.000000e+00,2016-02-04T00:00:00,2213 N 4th st,Day 3,POINT (-93.28554899999999 44.999581),5,Near North,Hawthorne,-93.285549,44.999581,02/04/2016,00:00
1,1002,-1.038496e+07,5.619481e+06,2016-02-04T00:00:00,821 Lyndale PL,Day 3,POINT (-93.28965794887922 44.98703965146753),5,Near North,Sumner - Glenwood,-93.289658,44.987040,02/04/2016,00:00
2,1003,-1.038620e+07,5.612199e+06,2016-02-04T00:00:00,3431 Humboldt Ave South,Day 3,POINT (-93.30085220376344 44.94075481079494),10,Calhoun Isle,ECCO,-93.300852,44.940755,02/04/2016,00:00
3,1004,-1.038058e+07,5.619614e+06,2016-02-04T00:00:00,521 3rd Ave se,Day 3,POINT (-93.25038048234984 44.98788273781837),3,University,Marcy Holmes,-93.250380,44.987883,02/04/2016,00:00
4,1005,-1.038060e+07,5.619218e+06,2016-02-04T00:00:00,321 4th ave se,Day 3,POINT (-93.25053762624376 44.98536724219078),3,University,Marcy Holmes,-93.250538,44.985367,02/04/2016,00:00


# Convert the Day column to an integer - to be uniform with the other data sets.

In [35]:
polk_tows.Day.unique()

array(['Day 3', 'Day 1', 'Day 2'], dtype=object)

In [36]:
polk_tows['Day'] = polk_tows.Day.map(lambda x: int(x.split()[1]))
polk_tows.head()

,FID,X,Y,Call_Taken,Location,Day,geometry,Ward,Community,Neighborhood,Longitude,Latitude,Date,Time
0,1001,0.000000e+00,0.000000e+00,2016-02-04T00:00:00,2213 N 4th st,3,POINT (-93.28554899999999 44.999581),5,Near North,Hawthorne,-93.285549,44.999581,02/04/2016,00:00
1,1002,-1.038496e+07,5.619481e+06,2016-02-04T00:00:00,821 Lyndale PL,3,POINT (-93.28965794887922 44.98703965146753),5,Near North,Sumner - Glenwood,-93.289658,44.987040,02/04/2016,00:00
2,1003,-1.038620e+07,5.612199e+06,2016-02-04T00:00:00,3431 Humboldt Ave South,3,POINT (-93.30085220376344 44.94075481079494),10,Calhoun Isle,ECCO,-93.300852,44.940755,02/04/2016,00:00
3,1004,-1.038058e+07,5.619614e+06,2016-02-04T00:00:00,521 3rd Ave se,3,POINT (-93.25038048234984 44.98788273781837),3,University,Marcy Holmes,-93.250380,44.987883,02/04/2016,00:00
4,1005,-1.038060e+07,5.619218e+06,2016-02-04T00:00:00,321 4th ave se,3,POINT (-93.25053762624376 44.98536724219078),3,University,Marcy Holmes,-93.250538,44.985367,02/04/2016,00:00


In [37]:
polk_tows_final = pd.DataFrame(polk_tows[['Date', 'Time', 'Location', 'Latitude', 'Longitude', 'Ward', 'Community', 'Neighborhood']])

In [38]:
polk_tows_final.head()

,Date,Time,Location,Latitude,Longitude,Ward,Community,Neighborhood
0,02/04/2016,00:00,2213 N 4th st,44.999581,-93.285549,5,Near North,Hawthorne
1,02/04/2016,00:00,821 Lyndale PL,44.987040,-93.289658,5,Near North,Sumner - Glenwood
2,02/04/2016,00:00,3431 Humboldt Ave South,44.940755,-93.300852,10,Calhoun Isle,ECCO
3,02/04/2016,00:00,521 3rd Ave se,44.987883,-93.250380,3,University,Marcy Holmes
4,02/04/2016,00:00,321 4th ave se,44.985367,-93.250538,3,University,Marcy Holmes


In [39]:
polk_tows_final['Emergency'] = 'Polk'
polk_tows_final.head()

,Date,Time,Location,Latitude,Longitude,Ward,Community,Neighborhood,Emergency
0,02/04/2016,00:00,2213 N 4th st,44.999581,-93.285549,5,Near North,Hawthorne,Polk
1,02/04/2016,00:00,821 Lyndale PL,44.987040,-93.289658,5,Near North,Sumner - Glenwood,Polk
2,02/04/2016,00:00,3431 Humboldt Ave South,44.940755,-93.300852,10,Calhoun Isle,ECCO,Polk
3,02/04/2016,00:00,521 3rd Ave se,44.987883,-93.250380,3,University,Marcy Holmes,Polk
4,02/04/2016,00:00,321 4th ave se,44.985367,-93.250538,3,University,Marcy Holmes,Polk


In [40]:
polk_tows_final.to_csv(os.path.join(OUTPUT_DIR, OUTPUT_FILE), index=False)

In [42]:
# Finally, remove the temporary file used
os.remove(os.path.join(OUTPUT_DIR, "polk_temp.csv"))